In [2]:
# Mount Google Drive


In [3]:
import json

jdj_path = '/content/drive/MyDrive/nbest_data/nbest_predictions_jdj.json'
psg_path = '/content/drive/MyDrive/nbest_data/nbest_predictions_psg.json'
kms_path = '/content/drive/MyDrive/nbest_data/pred_hardvoting.json'

with open(jdj_path, 'r') as f:
  nbest_pre_jdj = json.load(f)

with open(psg_path, 'r') as f:
  nbest_pre_psg = json.load(f)

with open(kms_path, 'r') as f:
  hard_voting_result = json.load(f)


In [7]:
nbest_val_1 = [x for x in nbest_pre_jdj.values()]
nbest_val_2 = [x for x in nbest_pre_psg.values()]
hard_voting_val = [x for x in hard_voting_result.values()]

In [4]:
def common_classfier_set(text_prob1, text_prob2):

  text_list = []
  text_list2 = []

  for tp in text_prob1:
    text_list.append(tp[0])
  for tp in text_prob2:
    text_list2.append(tp[0])

  common_text = list(set(text_list) & set(text_list2))
  print(f'common text : {common_text}')

  if len(common_text) < 1:
    print('There is No Common Predict!')
    text_prob1 = sorted(text_prob1, key=lambda x:x[1], reverse=True)
    text_prob2 = sorted(text_prob2, key=lambda x:x[1], reverse=True)
    common_text = max(text_prob1[0], text_prob2[0], key=lambda x:x[1])

    common_text = common_text[0].lstrip(' ')

    return [common_text]

  return common_text

In [5]:
def soft_voting(text_prob1, text_prob2, common_texts):

  prob = 0.0
  ensemble_results = []

  sum_text_prob = text_prob1 + text_prob2
  
  if len(common_texts) < 2:
      single_prob = 0
      single_prob2 = 0
      single_t =''
      flag = True
      flag2 = True

      for ct in common_texts:
        for tp in text_prob1:
          # print(tp)
          if ct in tp[0]:
            single_t = tp[0]
            single_prob = tp[1]
            flag = False
            break
        for tp in text_prob2:
          if ct in tp[0]:
            single_t = tp[0]
            single_prob2 = tp[1]
            flag2 = False
            break
      if flag != flag2:
        res = [single_t.lstrip(' '), max(single_prob,single_prob2)]
        print(f'공통아닌데 큰prob 넣어준 부분 : {res}')
        return [single_t.lstrip(' '), single_prob]
      else:
        text_prob1 = sorted(text_prob1, key=lambda x:x[1], reverse=True)    # 공통인데 max prob보다 한참 작은거
        text_prob2 = sorted(text_prob2, key=lambda x:x[1], reverse=True)
        maxi = max(text_prob1[0], text_prob2[0], key=lambda x:x[1])
        if max(single_prob,single_prob2) > maxi[1]:
          return [single_t.lstrip(' '), max(single_prob,single_prob2)]
        else:
          # return ensemble_results.append(maxi)
          maxi = [maxi[0].lstrip(' '), maxi[1]]
          return maxi

  for ct in common_texts:
    cnt = 1
    prob = 0.0
    for i in range(len(sum_text_prob)):
      if ct == sum_text_prob[i][0]:
        prob += sum_text_prob[i][1]
        cnt += 1
    # print(ct, prob , cnt)
    
    ensemble_results.append([ct, prob/cnt])

  ensemble_results = sorted(ensemble_results, key=lambda x:x[1], reverse=True)
  print(f'ensemble results : {ensemble_results}')

  if len(ensemble_results) < 1:
    print('empty')
  
  return ensemble_results[0]

In [6]:
def origin_top_nbest(text_prob1, text_prob2):
  tmp1 = sorted(text_prob1, key=lambda x:x[1], reverse=True)
  tmp2 = sorted(text_prob2, key=lambda x:x[1], reverse=True)

  print(tmp1)
  print(tmp2)

  a = tmp1[0]
  b = tmp2[0]
  return a, b

In [ ]:
k = 5
diff = 0

case1 = []
case2 = []

soft_voting_text_result=[]

for i in range(600):

  case1 = []
  case2 = []

  for idx in range(k):
    case1.append([nbest_val_1[i][idx]["text"].lstrip(' '), nbest_val_1[i][idx]["probability"]])
    case2.append([nbest_val_2[i][idx]["text"].lstrip(' '), nbest_val_2[i][idx]["probability"]])

  top_one, top_two = origin_top_nbest(case1, case2)
  common_text = common_classfier_set(case1, case2)
  sv_res = soft_voting(case1, case2, common_text)
  soft_voting_text_result.append(sv_res[0].lstrip(' '))
  # print(f'\n Top : {top_one},{top_two} \t\t Soft : {sv_res} \t\t Hard : {hard_voting_val[i]}\n')

  ## if sv_res[0].lstrip(' ') != hard_voting_val[i] and (sv_res[0].lstrip(' ') in hard_voting_val[i] or hard_voting_val[i] in sv_res[0].lstrip(' ')):
  if sv_res[0].lstrip(' ') != hard_voting_val[i]:
    diff += 1
    # print(f'\n****************  *******  ******************  ******************** ********************')
    # # print(f'- Top : {top_one},{top_two} \t Soft : {sv_res[1]} \t Hard : {hard_voting_val[i]}')
    # print(f'- 다름 \t\t index:{i} \t\t\t\t\t\t\t S:{sv_res[0]} \t H:{hard_voting_val[i]}')
    # print(f'****************  *******  ******************  ******************** ********************\n')
  # print('\n\n')
print(f'total diff --------------------------->  {diff}')

In [9]:
file_path = '/content/drive/MyDrive/nbest_data/predictions_final.json'

In [10]:
items = hard_voting_result.items()
key_id = []
for pair in items:
  key_id.append(pair[0])


In [11]:
len(key_id)

600

In [ ]:
stop_josa = ['의', '에', '는', '이', '을', '가','은','를']
def delete_josa(text):
  if '\n' in text:
    text = text.strip('\n').lstrip(' ').rstrip(' ')
  if text[-1] in stop_josa:
    text = text[:-1]
    return text.lstrip(' ').rstrip(' ')
  else:
    return text.lstrip(' ').rstrip(' ')

In [ ]:
all_predictions={}

for i in range(600):
  clean_text = delete_josa(soft_voting_text_result[i])
  all_predictions[key_id[i]] = clean_text

In [ ]:
with open(file_path, "w") as writer:
  writer.write(json.dumps(all_predictions, indent=4, ensure_ascii=False) + "\n")